In [319]:
import numpy as np

In [320]:
findp = np.array([0.2, 0.6, 0.4, 0.7])
transProb = np.array([[0, 0.4, 0.2, 0.4],
                      [0.6, 0, 0.4, 0],
                      [0.6, 0, 0, 0.4],
                      [0.4, 0.6, 0, 0]])
fares = np.array([[0, 25, 12, 6],
                [10, 0, 8, 0],
                [15, 0, 0, 9],
                [11, 5, 0, 0]])
costs = np.array([[0, 1, 1.5, 1.25],
                [1, 0, 1.75, 1],
                [1.5, 1, 0, 1.2],
                [1.25, 1, 1, 0]])
profits = fares - costs

In [331]:
v = np.zeros(4)
q = np.zeros([4, 4])
gamma = 0.95

for i in range(2):
    pickup = np.diag(findp*np.sum(transProb*(profits + gamma*v), axis = 1) + (1-findp)*gamma*v)
    move = -costs + gamma*v.reshape(4,1)
    np.fill_diagonal(move, 0)
    q = pickup + move
    v = np.max(q, axis=1)
    print(q, v)

print(np.argmax(q, axis=1))

[[ 2.72  -1.    -1.5   -1.25 ]
 [-1.     4.74  -1.75  -1.   ]
 [-1.5   -1.     4.488 -1.2  ]
 [-1.25  -1.    -1.     4.41 ]] [2.72  4.74  4.488 4.41 ]
[[5.653144 1.584    1.084    1.334   ]
 [3.503    8.494704 2.753    3.503   ]
 [2.7636   3.2636   8.33664  3.0636  ]
 [2.9395   3.1895   3.1895   8.28163 ]] [5.653144 8.494704 8.33664  8.28163 ]
[0 1 2 3]


In [322]:
from pulp import *

gamma = 0.95

model = LpProblem("Taxi", LpMinimize)
v1 = LpVariable("V(L1)", 0, None, LpContinuous)
v2 = LpVariable("V(L2)", 0, None, LpContinuous) 
v3 = LpVariable("V(L3)", 0, None, LpContinuous)
v4 = LpVariable("V(L4)", 0, None, LpContinuous)

model += v1 + v2 + v3 + v4

model += v1 >= 0.2*(0.4*(24 + gamma*v2) + 0.2*(10.5+gamma*v3) + 0.4*(4.75 + gamma*v4)) + 0.8*(gamma*v1)
model += v1 >= -1 + gamma*v2
model += v1 >= -1.5 + gamma*v3
model += v1 >= -1.25 + gamma*v4
model += v2 >= -1 + gamma*v1
model += v2 >= 0.6*(0.6*(9 + gamma*v1) + 0.4*(6.25 + gamma*v3)) + 0.4*(gamma*v2)
model += v2 >= -1.75 + gamma*v3
model += v2 >= -1 + gamma*v4
model += v3 >= -1.5 + gamma*v1
model += v3 >= -1 + gamma*v2
model += v3 >= 0.4*(0.6*(13.5 + gamma*v1) + 0.4*(7.8 + gamma*v4)) + 0.6*(gamma*v3)
model += v3 >= -1.2 + gamma*v4
model += v4 >= -1.25 + gamma*v1
model += v4 >= -1 + gamma*v2
model += v4 >= -1 + gamma*v3
model += v4 >= 0.7*(0.4*(9.75 + gamma*v1) + 0.6*(4 + gamma*v2)) + 0.3*(gamma*v4)

model.solve()

v = np.array([vi.varValue for vi in model.variables()])
print(v)

[67.8364 71.3913 71.5903 71.2442]


In [323]:
q = np.zeros([4, 4])
gamma = 0.95

for i in range(1):
    pickup = np.diag(findp*np.sum(transProb*(profits + gamma*v), axis = 1) + (1-findp)*gamma*v)
    move = -costs + gamma*v.reshape(4,1)
    np.fill_diagonal(move, 0)
    q = pickup + move

print(np.argmax(q, axis=1))

[0 1 2 3]
